In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from local_settings import postgresql as settings

In [2]:
import os
print(os.getcwd())

d:\Daniel\data_science\portfolio_projects\food_carbon_fp


In [3]:
def get_engine(user, passwd, host, port, db):
    url = f"mssql+pyodbc://{user}:{passwd}@{host},{port}/{db}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(url, pool_size= 50, echo= False)

    return engine

In [4]:
engine = get_engine(
    user= settings['pguser'],
    passwd=settings['pgpasswd'],
    host=settings['pghost'],
    port=settings['pgport'],
    db=settings['pgdb']
)

connection = engine.connect()

In [5]:
from sqlalchemy import inspect, MetaData, Table


inspector = inspect(engine)
print(inspector.get_table_names())

['foodprint_2018']


In [6]:
metadata = MetaData()

foodprint_2018 = Table('foodprint_2018', metadata, autoload= True, autoload_with= engine)

print(foodprint_2018.columns.keys())

['Id', 'Country', 'Product', 'Metric', 'Value']


In [8]:
from sqlalchemy import select
import pandas as pd

query = "SELECT * FROM foodprint_2018"

pd.read_sql_query(query, connection)

,Id,Country,Product,Metric,Value
0,1,Argentina,Pork,Supplied for Consumption (kg/person/year),10.51
1,2,Australia,Pork,Supplied for Consumption (kg/person/year),24.14
2,3,Albania,Pork,Supplied for Consumption (kg/person/year),10.88
3,4,Iceland,Pork,Supplied for Consumption (kg/person/year),21.69
4,5,New Zealand,Pork,Supplied for Consumption (kg/person/year),22.29
...,...,...,...,...,...
3245,126,Sierra Leone,Difference Animal Non-Animals,Products/person/year in kg,12.99
3246,127,Sri Lanka,Difference Animal Non-Animals,Products/person/year in kg,-4.34
3247,128,Indonesia,Difference Animal Non-Animals,Products/person/year in kg,-5.15
3248,129,Liberia,Difference Animal Non-Animals,Products/person/year in kg,-48.5


##### Exploratory Data Analysis

In [10]:
# Evaluated products and metrics

stmt = select(foodprint_2018.columns.Product.distinct(), foodprint_2018.columns.Metric)
stmt = stmt.order_by(foodprint_2018.columns.Product)

pd.DataFrame(connection.execute(stmt), 
             columns= [foodprint_2018.columns.keys()[1], foodprint_2018.columns.keys()[2]])

,Country,Product
0,Animal products,Total Kg CO2/year/person
1,Beef,Supplied for Consumption (kg/person/year)
2,Beef,Kg CO2/person/year
3,Difference Animal Non-Animals,Products/person/year in kg
4,Eggs,Supplied for Consumption (kg/person/year)
5,Eggs,Kg CO2/person/year
6,Fish,Supplied for Consumption (kg/person/year)
7,Fish,Kg CO2/person/year
8,Lamb & Goat,Supplied for Consumption (kg/person/year)
9,Lamb & Goat,Kg CO2/person/year


In [11]:
# Filtering information for the first country

query = """
    SELECT * 
        FROM foodprint_2018
        WHERE Country LIKE 'Albania'
        ORDER BY Product
"""

pd.read_sql_query(query, connection)

,Id,Country,Product,Metric,Value
0,3,Albania,Animal products,Total Kg CO2/year/person,1733.73
1,3,Albania,Beef,Supplied for Consumption (kg/person/year),22.5
2,3,Albania,Beef,Kg CO2/person/year,694.3
3,3,Albania,Difference Animal Non-Animals,Products/person/year in kg,1689.61
4,3,Albania,Eggs,Supplied for Consumption (kg/person/year),12.45
5,3,Albania,Eggs,Kg CO2/person/year,11.44
6,3,Albania,Fish,Supplied for Consumption (kg/person/year),3.85
7,3,Albania,Fish,Kg CO2/person/year,6.15
8,3,Albania,Lamb & Goat,Supplied for Consumption (kg/person/year),15.32
9,3,Albania,Lamb & Goat,Kg CO2/person/year,536.5


In [12]:
query = """
    SELECT *
        FROM foodprint_2018
        WHERE Product LIKE 'Fish'
"""

pd.read_sql_query(query, connection)

,Id,Country,Product,Metric,Value
0,1,Argentina,Fish,Supplied for Consumption (kg/person/year),4.36
1,2,Australia,Fish,Supplied for Consumption (kg/person/year),17.69
2,3,Albania,Fish,Supplied for Consumption (kg/person/year),3.85
3,4,Iceland,Fish,Supplied for Consumption (kg/person/year),74.41
4,5,New Zealand,Fish,Supplied for Consumption (kg/person/year),20.36
...,...,...,...,...,...
255,126,Sierra Leone,Fish,Kg CO2/person/year,51.14
256,127,Sri Lanka,Fish,Kg CO2/person/year,39.79
257,128,Indonesia,Fish,Kg CO2/person/year,40.95
258,129,Liberia,Fish,Kg CO2/person/year,6.59
